# TruGraph Tutorial: Instrumenting LangGraph Applications

This notebook demonstrates how to use TruGraph to instrument LangGraph applications for evaluation and monitoring.

## Overview

TruGraph provides:
- **Automatic detection** of LangGraph applications
- **Combined instrumentation** of both LangChain and LangGraph components
- **Multi-agent evaluation** capabilities
- **Automatic @task instrumentation** with intelligent attribute extraction

## Installation

First, make sure you have the required packages installed:


In [ ]:
# Install required packages
#!pip install trulens-apps-langgraph langgraph langchain-core langchain-openai langchain-community


## Basic Setup

Let's start by checking if LangGraph is available and importing the necessary components:


In [ ]:
# Check if LangGraph is available
try:
    from langgraph.graph import StateGraph, MessagesState, END
    from langchain_core.messages import HumanMessage, AIMessage
    from trulens.apps.langgraph import TruGraph
    from trulens.core.session import TruSession
    print("✅ LangGraph and TruGraph are available!")
    LANGGRAPH_AVAILABLE = True
except ImportError as e:
    print(f"❌ LangGraph not available: {e}")
    LANGGRAPH_AVAILABLE = False


## Example 1: Simple Multi-Agent Workflow

Let's create a basic multi-agent workflow with a researcher and writer:


In [ ]:
if LANGGRAPH_AVAILABLE:
    # Define agent functions
    def research_agent(state):
        """Agent that performs research on a topic."""
        messages = state.get("messages", [])
        if messages:
            last_message = messages[-1]
            if hasattr(last_message, "content"):
                query = last_message.content
            else:
                query = str(last_message)
            
            # Simulate research (in a real app, this would call external APIs)
            research_results = f"Research findings for '{query}': This is a comprehensive analysis of the topic."
            return {"messages": [AIMessage(content=research_results)]}
        
        return {"messages": [AIMessage(content="No research query provided")]}
    
    def writer_agent(state):
        """Agent that writes articles based on research."""
        messages = state.get("messages", [])
        if messages:
            last_message = messages[-1]
            if hasattr(last_message, "content"):
                research_content = last_message.content
            else:
                research_content = str(last_message)
            
            # Simulate article writing
            article = f"Article: Based on the research - {research_content[:100]}..."
            return {"messages": [AIMessage(content=article)]}
        
        return {"messages": [AIMessage(content="No research content provided")]}
    
    # Create the workflow
    workflow = StateGraph(MessagesState)
    workflow.add_node("researcher", research_agent)
    workflow.add_node("writer", writer_agent)
    workflow.add_edge("researcher", "writer")
    workflow.add_edge("writer", END)
    workflow.set_entry_point("researcher")
    
    # Compile the graph
    graph = workflow.compile()
    
    print("✅ Multi-agent workflow created successfully!")
    print(f"Graph type: {type(graph)}")
    print(f"Graph module: {graph.__module__}")
else:
    print("❌ Skipping workflow creation - LangGraph not available")


## Example 2: Test Auto-Detection

Let's test whether TruSession can automatically detect our LangGraph application:


## Automatic @task Detection

One of the key features of TruGraph is its ability to automatically detect and instrument functions decorated with LangGraph's `@task` decorator. This means you can use standard LangGraph patterns without any additional instrumentation code.

### How it works:

1. **Automatic Detection**: TruGraph automatically scans for functions decorated with `@task`
2. **Smart Attribute Extraction**: It intelligently extracts information from function arguments:
   - Handles `BaseChatModel` and `BaseModel` objects
   - Extracts data from dataclasses and Pydantic models
   - Skips non-serializable objects like LLM pools
   - Captures return values and exceptions
3. **Seamless Integration**: No additional decorators or code changes required

### Example Usage:

```python
from langgraph.func import task

@task  # This is automatically detected and instrumented by TruGraph
def my_agent_function(state, config):
    # Your agent logic here
    return updated_state
```

The instrumentation happens automatically when you create a TruGraph instance - no manual setup required!


In [ ]:
if LANGGRAPH_AVAILABLE:
    # Test the detection
    session = TruSession()
    
    print("🔍 Testing LangGraph Detection:")
    print(f"  Module check: {graph.__module__.startswith('langgraph')}")
    print(f"  Type check: {session._is_langgraph_app(graph)}")
    print(f"  Has graph attr: {hasattr(graph, 'graph')}")
    print(f"  Has invoke method: {hasattr(graph, 'invoke')}")
    
    # Test automatic detection
    print("\n🎯 Testing Automatic Detection:")
    tru_app = session.App(graph, app_name="AutoDetectionTest")
    
    print(f"  Created type: {type(tru_app)}")
    print(f"  Is TruGraph: {isinstance(tru_app, TruGraph)}")
    print(f"  App name: {tru_app.app_name}")
    
    if isinstance(tru_app, TruGraph):
        print("✅ SUCCESS: Auto-detection worked!")
        
        # Test basic functionality
        test_input = {"messages": [HumanMessage(content="What is AI?")]}
        result = tru_app.app.invoke(test_input)
        print(f"  Test result: {result['messages'][-1].content[:50]}...")
        
    else:
        print("❌ FAILED: Auto-detection didn't work")
        
    print("\n🎉 TruGraph Tutorial Complete!")
    print("For more examples, check the TruLens documentation at https://trulens.org/")
else:
    print("❌ Skipping detection test - LangGraph not available")
